## CKAN JSON Structure and Harvesting Sample Walkthrough Guide

- This guide will assist users in deconstructing the Comprehensive Knowledge Archive Network JavaScript Object Notation (CKAN JSON) site file structure and Application Programming Interface (API) design and how they are created through harvesting from selected portals.  
- A simple harvest sample will be presented to simulate the larger process and understand how the harvesting-metadata workflow is done.